### Collecting Libraries

In [1]:
import re
import pandas as pd
import numpy as np

# Pakai PySastrawi buat bantu stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Pakai nltk buat ambil stopwords nya
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Load to-be-cleansed file and supporting file

In [2]:
# Main file, di-define sebagai df_data
colnames = ['text', 'sentiment']
df_data = pd.read_csv('train_preprocess.tsv', sep='\t', header=None, names=colnames)
print('Done')

Done


In [3]:
# Kamus alay
kamus_label = ['alay', 'tidak alay']
kamus_alay = pd.read_csv('new_kamusalay.csv', header=None, encoding='latin-1', names=kamus_label)
print('Done')

Done


In [4]:
df_data.head()

,text,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [5]:
# Cek ada null atau nggak
df_data.isnull().sum()

text         0
sentiment    0
dtype: int64

In [6]:
# Iseng aja liat data seperti apa kalau didescribe
df_data.describe()

,text,sentiment
count,11000,11000
unique,10933,3
top,kesal,positive
freq,4,6416


In [7]:
# Mastiin lagi aja gak ada null
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       11000 non-null  object
 1   sentiment  11000 non-null  object
dtypes: object(2)
memory usage: 172.0+ KB


#### Stopwords

In [8]:
# Iseng aja liat list stopwordnya
list_stopword = stopwords.words('indonesian')
list_stopword

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [9]:
# Iseng juga liat panjang stopword
print(len(list_stopword))

758


In [10]:
# Extend stopword biar makin banyak, siapatau kurang
list_stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'nya', 'ber', 'banget', 'kali'])

In [11]:
list_stopword
print(len(list_stopword))

801


In [12]:
# Buat hilangin duplicated data di dalam list
list_stopword = list(dict.fromkeys(list_stopword))
print(len(list_stopword))

798


In [13]:
# List nya kita ubah jadi set
list_stopword = set(list_stopword)
list_stopword

{'&amp',
 'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'aja',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'amp',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banget',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'ber',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'b

In [14]:
# Ini dibuat soalnya sebelumnya data-data ini masuk ke dalam stopword nltk indonesian
bukan_stopword = {'baik', 'masalah', 'yakin', 'tidak', 'pantas', 'lebih'}

In [15]:
final_stopword = set([word for word in list_stopword if word not in bukan_stopword])

In [16]:
len(final_stopword)

792

In [17]:
# Iseng aja liat isi kamus alay
kamus_alay.head()

,alay,tidak alay
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [18]:
tes = re.findall('t3tapjokowi', str(kamus_alay['alay']))
print(tes)

['t3tapjokowi']


#### Data Cleansing :
- Normalizing words
- Stemming
- Removing Stopwords
- Final cleaning using RegEx

Membuat fungsi untuk melakukan cleaning

In [19]:
# Normalisasi kata alay (kalau ada), pake kamus alay
kamus_alay_map = dict(zip(kamus_alay['alay'], kamus_alay['tidak alay']))

# Bikin fungsi normalisasinya
def normalisasi_alay(text):
    return ' '.join([kamus_alay_map[word] if word in kamus_alay_map else word for word in text.split(' ')])

In [20]:
# Stemming pake PySastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Bikin fungsi stemming nya
def stemming(text):
    text = stemmer.stem(text)
    return text

In [21]:
# Bikin fungsi hilangin stopword
def remove_stopword(text):
    text = ' '.join(['' if word in final_stopword else word for word in text.split(' ')])
    text = text.strip()
    return text

In [22]:
# Bikin fungsi hilangin url pattern, whitespace berlebih, newline, tab, dan simbol-simbol
def cleaning(text):

    # Regex patern buat ilangin url nya di assign ke variabel url_pattern
    url_pattern = r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*'

    # Hilangin url_pattern pakai re.sub
    text = re.sub(url_pattern, " ", text)

    # Hilangin selain a-zA-Z0-9
    text = re.sub(r'[^a-zA-Z0-9]', " ", text)

    # Pisahin teks dan digit yang nyambung dalam satu string
    text = re.sub(r'(\d+)', r' \1 ', text)

    # Hilangin spasi 2x atau lebih
    text = re.sub(r' {2,}', " ", text)

    # Hilangin whitespace sama newline
    text = re.sub(r'\n\t',' ',text)

    # Hilangin kata 'user'
    text = re.sub(r'user',' ',text)

    # Ubah string jadi lowercase
    text = text.lower()
    return text

In [23]:
# Pakai fungsi normalisasi alay untuk seluruh data dalam df_data
for data in [df_data]:
    data['text_normalized'] = data['text'].apply(normalisasi_alay)
print('Done')

Done


In [24]:
idx = 787
print(df_data['text'][idx])
print(df_data['text_normalized'][idx])

kebaikan
kebaikan


In [25]:
# Pakai fungsi stemming untuk seluruh data dalam df_data
for data in [df_data]:
    data['text_stem'] = data['text_normalized'].apply(stemming)
print('Done')

Done


In [26]:
print(df_data['text_normalized'][idx])
print(df_data['text_stem'][idx])

kebaikan
baik


In [27]:
# Pakai fungsi hilangin stopword untuk seluruh data dalam df_data
for data in [df_data]:
    data['text_wsw'] = data['text_stem'].apply(remove_stopword)
print('Done')

Done


In [28]:
print(df_data['text_stem'][idx])
print(df_data['text_wsw'][idx])

baik
baik


In [29]:
# Pakai fungsi cleaning untuk seluruh data dalam df_data
for data in [df_data]:
    data['text_clean'] = data['text_wsw'].apply(cleaning)
print('Done')

Done


In [30]:
print(df_data['text_wsw'][idx])
print(df_data['text_clean'][idx])

baik
baik


In [31]:
# Mengurutkan kolom biar enak dilihat
df_data = df_data[['text', 'text_normalized', 'text_stem', 'text_wsw', 'text_clean', 'sentiment']]
df_data.head()

,text,text_normalized,text_stem,text_wsw,text_clean,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,warung ini dimiliki oleh pengusaha pabrik tahu...,warung ini milik oleh usaha pabrik tahu yang s...,warung milik usaha pabrik puluh kenal ...,warung milik usaha pabrik puluh kenal putih ba...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus k212 mmbri hujjah partai ...,mohon ulama lurus k 212 mmbri hujjah partai di...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,lokasi strategis di jalan sumatra bandung . te...,lokasi strategis di jalan sumatra bandung temp...,lokasi strategis jalan sumatra bandung nyam...,lokasi strategis jalan sumatra bandung nyaman ...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia unboxing paket barang bag...,betapa bahagia unboxing paket barang bagus beli,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,aduh . jadi mahasiswa jangan sombong dong . ka...,aduh jadi mahasiswa jangan sombong dong kasih ...,aduh mahasiswa sombong kasih kartu kuning ...,aduh mahasiswa sombong kasih kartu kuning ajar...,negative


In [32]:
# Compare text awal banget vs yang sudah lewatin semua tahap cleansing
print(df_data['text'][idx])
print(df_data['text_clean'][idx])

kebaikan
baik


### Save new csv file for clean text

In [33]:
# Drop dulu kolom yang udah gak kepake biar gak kebanyakan
df_data = df_data.drop(['text', 'text_normalized', 'text_stem', 'text_wsw'], axis=1)

In [34]:
# Iseng cek dataframe baru nya
df_data.head()

,text_clean,sentiment
0,warung milik usaha pabrik puluh kenal putih ba...,positive
1,mohon ulama lurus k 212 mmbri hujjah partai di...,neutral
2,lokasi strategis jalan sumatra bandung nyaman ...,positive
3,betapa bahagia unboxing paket barang bagus beli,positive
4,aduh mahasiswa sombong kasih kartu kuning ajar...,negative


In [35]:
print(df_data['text_clean'][787])

baik


In [36]:
# Save dataframe nya ke csv lagi
df_data.to_csv('text_clean.csv', index=False)

### Preprocess selesai di sini